# Error propagation with MCMC

In [1]:
import os
import glob
import pickle
import datetime
# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.special
# Library to perform MCMC runs
import emcee

import mwc_induction_utils as mwc

# Useful plotting libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import corner

# favorite Seaborn settings for notebooks
rc={'lines.linewidth': 2, 
    'axes.labelsize' : 16, 
    'axes.titlesize' : 18,
    'axes.facecolor' : 'F4F3F6',
    'axes.edgecolor' : '000000',
    'axes.linewidth' : 1.2,
    'xtick.labelsize' : 13,
    'ytick.labelsize' : 13,
    'grid.linestyle' : ':',
    'grid.color' : 'a6a6a6'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)
sns.set_palette("deep", color_codes=True)

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables SVG graphics inline (only use with static plots (non-Bokeh))
%config InlineBackend.figure_format = 'svg'

# Generate a variable with the day that the script is run
today = str(datetime.datetime.today().strftime('%Y%m%d'))

# Defining the problem

In our first parameter estimation [using MCMC](https://github.com/RPGroup-PBoC/mwc_induction/blob/master/code/analysis/MCMC_parameter_estimation.ipynb) we reported a credible region for the MWC parameters and the fold-change based on the data and the fit to the model. But these estimates didn't include previous characterized uncertainty in the parameters of the model.

For example at the time we computed the fold-chage for each of the RBS mutants we assumed we knew with 100% certainty the mean repressor copy number. This assumption is far from the truth since in their paper [Garcia and Phillips](http://www.pnas.org/content/108/29/12173.abstract) report the mean $\pm$ standard deviation of the repressor copy number as revealed by multiple measurements of these quantities. The same applies to the repressor binding energies.

The question then becomes how do we include these sources of uncertainty into our fold-change model with induction?

## Bayes theorem as a method for learning

The Bayesian framework gives a natural form on how to solve this issue. Let's focus first on the uncertainty of the repressor. We could write our parameter estimation using Bayes theorem as

\begin{equation}
P(\epsilon_A, \epsilon_I, R \mid D, I) \propto P(D \mid \epsilon_A, \epsilon_I, R, I) \cdot P(\epsilon_A, \epsilon_I, R \mid I),
\end{equation}

where we explicitly included the dependence on the repressor copy number. The second term on the right hand side, the so-called prior, **includes all the information we know before performing the experiment**. For simplicity we can assume that the parameters are independent such that we can rewrite this term as

\begin{equation}
P(\epsilon_A, \epsilon_I, R \mid I) = P(\epsilon_A \mid I) \cdot P(\epsilon_I \mid I) \cdot P(R \mid I).
\end{equation}

Usually one doesn't want to bias the inference with the prior, therefore we are train to use *maximally uniformative priors* for the parameters. So for our previous model we chose a Jeffreys' prior for the $\epsilon_A$ and $\epsilon_I$ parameteres, i.e.

\begin{align}
P(\epsilon_A \mid I) &\propto \frac{1}{\epsilon_A}, \\
P(\epsilon_I \mid I) &\propto \frac{1}{\epsilon_I}
\end{align}

But in a sense Bayes theorem is a *model for learning*. What we mean with that is that it naturally allows us to update our parameter estimates after performing an experiment and obtaining data. So in the case when one doesn't know anything at all about the parameter value these uniformative priors are the right thing to use. Now, what happens when one does indeed have information about the value of some of these parameters from previous experiments? Well, then that prior information should be included in the prior!

### Prior on the parameters to include sources of uncertainty

For the term $P(R \mid I)$ we should include the characterized uncertainty that Garcia and Phillips reported in their paper. Quoting the paper it reads
> Immunoblots were used to measure the number of Lac repressors in six strains with different constitutive levels of Lac repressor. Each value corresponds to an average of cultures grown on at least 3 different days. The error bars are the SD of these measurements.

This means that the number they report is the *mean repressor copy number* and the *standard deviation on these measurements*. It is important to clarify that this standard deviation **does not** reflect the single-cell variability of repressors, but the experimental uncertainty when measuring the mean repressor copy number. In other words this standard deviation captures the lack of perfect accuracy when measuring this parameter, not the natural biological noise one expects on a clonal population.

The repressor copy number per cell itself is a *discrete variable*, therefore one could naively think that the prior should be therefore given by a discrete distribution. But we should recall that what Garcia and Phillips measured with their immunoblots was not a single cell measurement, but a bulk measurement where they got to measure the **mean repressor copy number** which is not a discrete variable itself. As a matter of fact by the central limit theorem we know that we expect the distribution of the mean repressor copy number to be Gaussian. Therefore we can write

\begin{equation}
P(R \mid \sigma_R, I) = \frac{1}{\sqrt{2 \pi \sigma_R^2}}\exp \left[ \frac{(R - R^*)^2}{2 \sigma_R^2}  \right],
\end{equation}

where $\sigma_R$ is the characterized standard deviation that Garcia and Phillips experimentally characterized for each RBS mutant, and $R^*$ is the mean repressor level also characterized experimentally.

If we include this prior for the repressor copy number and an equivalent one for the repressor binding energy, and allow the MCMC walkers to walk on these two new dimensions while fitting the MWC parameters, then the posterior probability for the fold-change would include all of the characterized uncertainty on the parameters! In this way we can properly build the credible region for our predictions.

### The log posterior probability

Including the uncertainty on the mean repressor copy number and the binding energy implies that our posterior is now of the form

\begin{equation}
P(\epsilon_A, \epsilon_I, R_i, \epsilon_{r_j} \mid fc_{\exp_{ijk}}, R_i^*, \sigma_{R_i}, \epsilon_{r_j}^*, \sigma_{\epsilon_{r_j}}, C_k, I) \propto \prod_i \prod_j \prod_k P(fc_{\exp_{ijk}} \mid \epsilon_A, \epsilon_I, R_i, \epsilon_{r_j}, R_i^*, \sigma_{R_i}, \epsilon_{r_j}^*, \sigma_{\epsilon_{r_j}}, C_k, I) \cdot P(\epsilon_A \mid C_k, I) \cdot P(\epsilon_I \mid C_k, I) \cdot P(R_i \mid R_i^* \sigma_{R_i}, C_k, I) \cdot P(\epsilon_{r_j} \mid \epsilon_{r_j}^*, \sigma_{\epsilon_{r_j}}, C_k, I),
\end{equation}

where the subindex $i$ lists all the RBS mutants, the subindex $j$ lists all the operators, and the subindex $k$ lists all the IPTG concentrations $C$. $fc_{\exp_{ijk}}$ represents the experimentally determined fold-change.

This form of the posterior means that the parameter estimation will be done over the two MWC parameters, however many RBS mutants are included and also however many operators. The independent variables now become:
1. The IPTG concentration.
2. The experimental mean repressor copy number.
3. The experimental standard deviation on the repressor copy number.
4. The experimental mean repressor binding energy.
5. the experimental standard deviation on the repressor binding enery.

The function functional form of the log posterior probability assuming as before a Gaussian distribution of the residuals is of the form

\begin{equation}
\ln P(\epsilon_A, \epsilon_I, R_i, \epsilon_{r_j} \mid fc_{\exp_{ijk}}, R_i^*, \sigma_{R_i}, \epsilon_{r_j}^*, \sigma_{\epsilon_{r_j}}, C_k, \sigma, I) \propto - (n + 1) \ln \sigma - \ln \epsilon_A - \ln \epsilon_I + \sum_i \sum_j \sum_k  - \frac{1}{2 \sigma^2} \left(fc_{\exp_{ijk}} - fc(\epsilon_A, \epsilon_I, R_i, \epsilon_{r_j}, C_k) \right)^2 - \frac{\left( R_i - R_i^* \right)^2}{2 \sigma^2_{R_i}} - \frac{\left( \epsilon_{r_j} - \epsilon_{r_j}^* \right)^2}{2 \sigma^2_{\epsilon_{r_j}}} ,
\end{equation}

where $n$ is the total number of data points.

In [80]:
def mcmc_pre_process(df):
    """
    Pre-process the tidy DataFrame to prepare it for the MCMC. This is done
    separately from the log-posterior calculation to speed up the process
    avoiding parsing the DataFrame every evaluation of the posterior.
    Parameteres
    -----------

    Returns
    -------
    
    """
    # List the unique variables
    rep_unique = np.sort(df.repressors.unique())
    eps_unique = np.sort(df.binding_energy.unique())
    IPTG_unique = np.sort(df.IPTG_uM.unique())
    
    # determine the number of unique variables
    n_repressor = len(rep_unique)
    n_epsilon_r = len(eps_unique)
    n_IPTG = len(IPTG_unique)
    
    # Depending on the number of parameters determine the indexes of the
    # parameters to fit
    param_idx = np.cumsum([3, n_repressor, n_epsilon_r])
    
    # Sort the data frame such that the log-posterior function can
    # automatically compute the log probability with the right parameters
    # for each data point
    df_sort = df.sort(['repressors', 'binding_energy', 'IPTG_uM'])
    data = np.array(df_sort[['fold_change_A', 'IPTG_uM', 
                             'repressors', 'delta_repressors', 
                             'binding_energy', 'delta_energy']])
    return [rep_unique, eps_unique], param_idx, data

In [ ]:
def log_likelihood(param, param_idx, unique_var, data, epsilon=4.5):
    '''
    Returns the log-likelihood
    Parameters
    ----------
    
    Returns
    -------
    '''
    # unpack parameters
    ea, ei, sigma = param[0:param_idx[0]] # MWC parameters
    rep = param[idx[0]:param_idx[1]] # Repressor copy numbers
    eps_r = param[idx[1]:param_idx[2]] # Represor energies
   
    # Initialize the log_likelihood
    log_like = 0
    # loop through the parameters to fit in order to compute the
    # theoretical fold change using the right parameters for each strain
    for i, r in enumerate(unique_var[0]):
        for j, eps in enumerate(unique_var[1]):
            data_block = data[(data[:, 2]==r) & (data[:, 4]==eps), :]
            # compute the theoretical fold-change
            fc_theory = mwc.fold_change_log(IPTG=data_block[:, 1], 
                                            ea, ei, epsilon, 
                                            rep[i], eps_r[j])
            # compute the log likelihood for this block of data
            log_like +=  np.sum((fc_theory - data_block[:, 0])**2) / 2 / sigma**2

    return log_like


# def log_post(param, df, epsilon=4.5):
#     '''
#     Computes the log posterior probability.
#     Parameters
#     ----------
#     param : array-like.
#         The parameters to be fit by the MCMC. This must be an array of length 3
#         with the following entries
#         param[0] = ea == -lnKa
#         param[1] = ei == -lnKi
#         param[2] = sigma. Homoscedastic error associated with the Gaussian 
#         likelihood.
#     indep_var : n x 3 array.
#         Series of independent variables to compute the theoretical fold-change.
#         1st column : IPTG concentration
#         2nd column : repressor copy number
#         3rd column : repressor binding energy
#     dep_var : array-like
#         Dependent variable, i.e. experimental fold-change. Then length of this
#         array should be the same as the number of rows in indep_var.
#     ea_range : array-like.
#         Range of variables to use in the prior as boundaries for the ea parameter.
#     ei_range : array-like.
#         Range of variables to use in the prior as boundaries for the ei parameter.
#     sigma_range : array-like.
#         Range of variables to use in the prior as boundaries for the sigma param.
#     epsilon : float.
#         Energy difference between the active and inactive state of the repressor.
#     '''
    
    
#     # unpack parameters
#     ea, ei, sigma = param
    
#     # Set the prior boundaries. Since the variables have a Jeffreys prior, in
#     # the log probability they have a uniform prior
#     if ea > np.max(ea_range) or ea < np.min(ea_range)\
#     or ei > np.max(ei_range) or ea < np.min(ei_range)\
#     or sigma > np.max(sigma_range) or sigma < np.min(sigma_range):
#         return -np.inf
    
#     return -(len(indep_var) + 1) * np.log(sigma) \
#     - log_likelihood(param, indep_var, dep_var, epsilon)

In [42]:
# List the error sources as described by Garcia & Phillips PNAS 2011.
delta_R = {'HG104':2, 'RBS1147':10, 'RBS446':15, 'RBS1027':20, 'RBS1':80,
               'RBS1L':170}
delta_epsilon_r = {'O1':0.2, 'O2':0.2, 'O3':0.1, 'Oid':0.2}

In [55]:
datadir = '../../data/'
# read the list of data-sets to ignore
data_ignore = pd.read_csv(datadir + 'datasets_ignore.csv', header=None).values
# read the all data sets except for the ones in the ignore list
all_files = glob.glob(datadir + '*' + 'O2_IPTG_titration' + '*csv')
ignore_files = [f for f in all_files for i in data_ignore if i[0] in f]
read_files = [f for f in all_files if f not in ignore_files]
print('Number of unique data-sets: {:d}'.format(len(read_files)))
df = pd.concat(pd.read_csv(f, comment='#') for f in read_files)

# Now we remove the autofluorescence and delta values
df = df[(df.rbs != 'auto') & (df.rbs != 'delta')]
# Restart index
df = df.reset_index()

# Add the error columns to the data frame
df['delta_repressors'] = pd.Series([delta_R[df.iloc[x].rbs] for x\
                                    in np.arange(df.shape[0])])
df['delta_energy'] = pd.Series([delta_epsilon_r[x] for x in df.operator])

df.head()

Number of unique data-sets: 11


,index,date,username,operator,binding_energy,rbs,repressors,IPTG_uM,mean_YFP_A,mean_YFP_bgcorr_A,fold_change_A,delta_repressors,delta_energy
0,2,20160804,mrazomej,O2,-13.9,RBS1L,870,0.0,3624.474605,111.851286,0.007146,170,0.2
1,3,20160804,mrazomej,O2,-13.9,RBS1,610,0.0,3619.786265,107.162946,0.006847,80,0.2
2,4,20160804,mrazomej,O2,-13.9,RBS1027,130,0.0,3717.019527,204.396208,0.013059,20,0.2
3,5,20160804,mrazomej,O2,-13.9,RBS446,62,0.0,3854.650585,342.027265,0.021853,15,0.2
4,6,20160804,mrazomej,O2,-13.9,RBS1147,30,0.0,4169.802851,657.179531,0.041988,10,0.2


In [58]:
df_sort = df.sort(['repressors', 'binding_energy', 'IPTG_uM'])
np.array(df_sort[['repressors', 'delta_repressors', 
                  'binding_energy', 'delta_energy',
                  'IPTG_uM']]).shape

/Users/razo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


(774, 5)